In [1]:
import os
import tempfile

from csimpy import core as csim

from biosimulators_utils.config import get_config
from biosimulators_utils.report import data_model as report_data_model
from biosimulators_utils.viz import data_model as viz_data_model

In [2]:
config = get_config()
config.REPORT_FORMATS = [report_data_model.ReportFormat.csv]
config.VIZ_FORMATS = [viz_data_model.VizFormat.png]
config.BUNDLE_OUTPUTS = False
config.KEEP_INDIVIDUAL_OUTPUTS = True
config.LOG = False
config.VERBOSE = False

In [3]:
sedml_root = os.path.join('tests', 'fixtures')
sedml_doc = os.path.join(sedml_root, 'sine_approximations_import.sedml')
output_root = tempfile.mkdtemp()

In [6]:
%%capture execution_log
results, log = csim.exec_sed_doc(sedml_doc, sedml_root, output_root, config=config)

SedmlExecutionError: The SED document did not execute successfully:

  CSimPy failed to instaniate model.

  CSimPy failed to instaniate model.

In [7]:
sedml_root = os.path.join('..', 'diabetic-wound-healing', '7ef')
sedml_doc = os.path.join(sedml_root, 'waugh_sherratt_2006.sedml')
output_root = os.path.join('waugh_sherratt_2006.sedml')

In [8]:
%%capture execution_log
results, log = csim.exec_sed_doc(sedml_doc, sedml_root, output_root, config=config)

SedmlExecutionError: The SED document did not execute successfully:

  CSimPy failed to instaniate model.

  CSimPy failed to instaniate model.

  CSimPy failed to instaniate model.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
plot1 = pd.read_csv('waugh_sherratt_2006.sedml/plot1.csv', index_col=0, header=None).T
axs = plot1.plot.line(y='base_model_phi_I', x='base_model_time', label='normal')
plot1.plot.line(y='diabetic_model_phi_I', x='diabetic_model_time', label='diabetic', ax=axs)
axs.set_ylabel("$\phi_I$ [cells/mm$^3$]");
axs.set_xlabel("time [day]");

In [ ]:
plot2 = pd.read_csv('waugh_sherratt_2006.sedml/plot2.csv', index_col=0, header=None).T
axs = plot2.plot.line(y='base_model_phi_R', x='base_model_time', label='normal')
plot2.plot.line(y='diabetic_model_phi_R', x='diabetic_model_time', label='diabetic', ax=axs)
axs.set_ylabel("$\phi_R$ [cells/mm$^3$]");
axs.set_xlabel("time [day]");

In [ ]:
plot3 = pd.read_csv('waugh_sherratt_2006.sedml/plot3.csv', index_col=0, header=None).T
axs = plot3.plot.line(y='base_model_TGFb', x='base_model_time', label='normal')
plot3.plot.line(y='diabetic_model_TGFb', x='diabetic_model_time', label='diabetic', ax=axs)
axs.set_ylabel("TGF-$\\beta$ [pg/mm$^3$]");
axs.set_xlabel("time [day]");

In [ ]:
plot4 = pd.read_csv('waugh_sherratt_2006.sedml/plot4.csv', index_col=0, header=None).T
axs = plot4.plot.line(y='diabetic_total_macrophage', x='diabetic_model_time', label='untreated')
plot4.plot.line(y='diabetic_with_treatment_total_macrophage', x='diabetic_model_with_treatment_time', label='treated', ax=axs)
axs.set_ylabel("Total macrophage density [cells/mm$^3$]");
axs.set_xlabel("time [day]");

In [ ]:
personal_data = {
    'cellml_model': os.path.abspath(os.path.join(sedml_root, 'waugh_sherratt_2006.cellml')),
    'alpha': 0.5,
    'alpha_post_treatment': 0.2,
    'treatment_time': 10
}

from string import Template
sedml_template = os.path.join(sedml_root, 'personalised-treatment.sedml.template')
with open(sedml_template) as f:
    tmpl = f.read()
s = Template(tmpl).substitute(personal_data)
personalised_sedml_doc = os.path.join(output_root, 'personalised-treatment.sedml')
with open(personalised_sedml_doc, 'w') as f:
    f.write(s)
    f.close()



In [ ]:
%%capture execution_log
personalised_output_root = os.path.join(output_root, 'personalised')
results, log = csim.exec_sed_doc(personalised_sedml_doc, sedml_root, personalised_output_root, config=config)

In [ ]:
csv = os.path.join(personalised_output_root, 'plot1.csv')
plot1 = pd.read_csv(csv, index_col=0, header=None).T
axs1 = plot1.plot.line(y='diabetic_model_phi_I', x='diabetic_model_time', label='diabetic')
plot1.plot.line(y='p_diabetic_model_phi_I', x='p_diabetic_model_time', label='personalised', ax=axs1)
axs1.set_ylabel("$\phi_I$ [cells/mm$^3$]");
axs1.set_xlabel("time [day]");
csv = os.path.join(personalised_output_root, 'plot2.csv')
plot2 = pd.read_csv(csv, index_col=0, header=None).T
axs2 = plot2.plot.line(y='diabetic_model_phi_R', x='diabetic_model_time', label='diabetic')
plot2.plot.line(y='p_diabetic_model_phi_R', x='p_diabetic_model_time', label='personalised', ax=axs2)
axs2.set_ylabel("$\phi_R$ [cells/mm$^3$]");
axs2.set_xlabel("time [day]");
csv = os.path.join(personalised_output_root, 'plot3.csv')
plot3 = pd.read_csv(csv, index_col=0, header=None).T
axs3 = plot3.plot.line(y='diabetic_model_TGFb', x='diabetic_model_time', label='diabetic')
plot3.plot.line(y='p_diabetic_model_TGFb', x='p_diabetic_model_time', label='personalised', ax=axs3)
axs3.set_ylabel("TGF-$\\beta$ [pg/mm$^3$]");
axs3.set_xlabel("time [day]");
csv = os.path.join(personalised_output_root, 'plot4.csv')
plot4 = pd.read_csv(csv, index_col=0, header=None).T
axs4 = plot4.plot.line(y='diabetic_total_macrophage', x='diabetic_model_time', label='diabetic untreated')
plot4.plot.line(y='diabetic_with_treatment_total_macrophage', x='diabetic_model_with_treatment_time', label='diabetic treated', ax=axs4)
plot4.plot.line(y='p_diabetic_total_macrophage', x='p_diabetic_model_time', label='personalised untreated', ax=axs4)
plot4.plot.line(y='p_diabetic_with_treatment_total_macrophage', x='p_diabetic_model_with_treatment_time', label='personalised treated', ax=axs4)
axs4.set_ylabel("Total macrophage density [cells/mm$^3$]");
axs4.set_xlabel("time [day]");